In [1]:
import requests
import json
import pandas as pd

In [2]:
#Due to the limits of the amount of results the API can return, query must be performed in loop by date

In [3]:
base_url = "http://www.patentsview.org/api/patents/query?"
query = "q={\"_and\":[{\"patent_type\":\"Design\"},{\"_gte\":{\"app_date\":\"1980-01-01\"}},{\"_lte\":{\"app_date\":\"2015-12-31\"}}, {\"patent_firstnamed_inventor_country\":\"US\"}]}"
field_list = "&f=[\"patent_number\",\"app_date\",\"uspc_mainclass_id\"]"
options = "&o={\"per_page\":50,\"page\":1}"

In [4]:
full_url = base_url + query + field_list + options

In [5]:
full_url

'http://www.patentsview.org/api/patents/query?q={"_and":[{"patent_type":"Design"},{"_gte":{"app_date":"1980-01-01"}},{"_lte":{"app_date":"2015-12-31"}}, {"patent_firstnamed_inventor_country":"US"}]}&f=["patent_number","app_date","uspc_mainclass_id"]&o={"per_page":50,"page":1}'

In [6]:
r = requests.get(full_url)

In [7]:
data = r.json()

In [8]:
data['patents']

[{'applications': [{'app_date': '1982-03-02', 'app_id': '06/353997'}],
  'patent_number': '4451875',
  'uspcs': [{'uspc_mainclass_id': '362'}, {'uspc_mainclass_id': '40'}]},
 {'applications': [{'app_date': '1984-09-24', 'app_id': '06/653170'}],
  'patent_number': '4544582',
  'uspcs': [{'uspc_mainclass_id': '427'}, {'uspc_mainclass_id': '428'}]},
 {'applications': [{'app_date': '1984-07-10', 'app_id': '06/629371'}],
  'patent_number': '4547613',
  'uspcs': [{'uspc_mainclass_id': '502'}, {'uspc_mainclass_id': '585'}]},
 {'applications': [{'app_date': '1982-09-07', 'app_id': '06/415757'}],
  'patent_number': '4568335',
  'uspcs': [{'uspc_mainclass_id': '604'}]},
 {'applications': [{'app_date': '1984-07-24', 'app_id': '06/634143'}],
  'patent_number': '4583454',
  'uspcs': [{'uspc_mainclass_id': '126'},
   {'uspc_mainclass_id': '34'},
   {'uspc_mainclass_id': '99'}]},
 {'applications': [{'app_date': '1984-09-28', 'app_id': '06/655517'}],
  'patent_number': '4589463',
  'uspcs': [{'uspc_ma

In [9]:
df = pd.io.json.json_normalize(data['patents'])

In [10]:
def clean_dataframe(df):
    """ Cleans dataframe created by PatentsView JSON object
        
        Args:
        df(dataframe): the datafame to be cleaned
        
        Returns:
        df(dataframe): the proccessed dataframe
    """
    
    df = df[df.patent_number.str.contains('D')]

    df['app_date'] = df['applications'].astype(str).str.extract('(\d{4}-\d{2}-\d{2})')
    pd.to_datetime(df.app_date, errors='coerce')
    df.drop('applications', axis=1, inplace=True)

    df['uspcs'] = df['uspcs'].astype(str).str.findall('(\d{3}|D\d{2})')
    holder = df.apply(lambda x: pd.Series(x['uspcs']),axis=1).stack().reset_index(level=1, drop=True)
    holder.name = 'class'
    return df.drop('uspcs', axis=1).join(holder)


In [11]:
test = clean_dataframe(df)

/home/tiera/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  del sys.path[0]
/home/tiera/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/tiera/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/tiera/anaconda3/li

In [12]:
test

,patent_number,app_date,class
7,D257752,1980-01-14,D19
8,D257924,1980-01-14,D06
9,D258382,1980-01-17,D23
10,D258383,1980-01-17,D23
11,D258571,1980-04-03,D09
12,D258650,1980-06-09,D12
13,D258678,1980-01-17,D23
14,D258755,1980-04-01,D23
15,D258760,1980-02-28,D23
16,D258766,1980-01-28,D25
